In [2]:
# Replace spaces with commas in a text file
input_file = 'marvin.txt'
output_file = 'marvin_copper.txt'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Split by any whitespace and join with commas
        fields = line.strip().split()
        outfile.write(','.join(fields) + '\n')


In [3]:
import pandas as pd
import numpy as np

# Load the file with no header
df = pd.read_csv("marvin_copper.txt", header=None)

# Rename columns
df.columns = ["id", "i", "j", "k", "tonne", "au_ppm", "cu_pct", "proc_profit"]

df["id+1"] = df["id"] + 1

# convert ijk to centroid with block size = 30

# Define block size and origin
block_size = 30
origin = (0, 0, 0)  # Modify if needed

# Compute centroid coordinates
df["X"] = origin[0] + (df["i"] + 0.5) * block_size
df["Y"] = origin[1] + (df["j"] + 0.5) * block_size
df["Z"] = origin[2] + (df["k"] + 0.5) * block_size


# Calculate block value

Rec_CU = 0.88
Rec_AU = 0.6

Price_AU = 12 # $/g
Selling_AU = 0.2

Price_CU = 20 * (2204/100) # assumed this is cents/lb since its not stated in the documentation. convert to $/t
Selling_CU = 7.2 * (2204/100)

m_cost = 0.9 # mining cost $/t
p_cost = 4.0 # processing cost $/t

df["rev_temp"] = df["tonne"] * ((df["au_ppm"] * Rec_AU * (Price_AU - Selling_AU)) + ((df["cu_pct"]/100) * Rec_CU * (Price_CU - Selling_CU)))
df["mine_cost"] = (df["tonne"] * m_cost)
df["proc_cost"] = (df["tonne"] * p_cost)

# temporary block value
df["val_temp"] = df["rev_temp"] - df["mine_cost"] - df["proc_cost"]

# if val_temp is negative, the block will not be processed, so revenue = 0
df["rev"] = np.where(df["val_temp"] > 0, df["rev_temp"], 0)

df["block_val"] = df["rev"] - df["mine_cost"] - df["proc_cost"]


# Add slope

# Calculate midpoints
x_mid = df["X"].median()
y_mid = df["Y"].median()

# Initialize slope column with NaN
df["slope"] = np.nan

# Apply conditions for each quadrant
df.loc[(df["X"] > x_mid) & (df["Y"] > y_mid), "slope"] = 40  # Q1
df.loc[(df["X"] <= x_mid) & (df["Y"] > y_mid), "slope"] = 60  # Q2
df.loc[(df["X"] <= x_mid) & (df["Y"] <= y_mid), "slope"] = 50  # Q3
df.loc[(df["X"] > x_mid) & (df["Y"] <= y_mid), "slope"] = 45  # Q4

#print(df["slope"].value_counts())

# Reorder
df = df[["id+1", "X", "Y", "Z", "tonne", "au_ppm", "cu_pct", "block_val", "slope"]]

# Save to new file
df.to_csv("marvin_copper_final.csv", index=False)


In [4]:
print(df["slope"].value_counts())

slope
50.0    14234
60.0    13200
45.0    13073
40.0    12764
Name: count, dtype: int64


In [ ]:
print("Min:", df["rev_temp"].min())
print("Max:", df["rev_temp"].max())

print("Min:", df["val_temp"].min())
print("Max:", df["val_temp"].max())

print("Min:", df["cu_pct"].min())
print("Max:", df["cu_pct"].max())

print("Min:", df["block_val"].min())
print("Max:", df["block_val"].max())

In [38]:
import plotly.express as px

# Sample 3D scatter plot
fig = px.scatter_3d(
    df,
    x="X",
    y="Y",
    z="Z",
    color="slope",
    size_max=5,
    opacity=1,
    title="3D Block Model Colored by Slope",
    hover_data=["id", "slope", "block_val"]  # Optional, customize as needed
)

# Customize layout
fig.update_traces(marker=dict(size=4))
fig.update_layout(scene=dict(
    xaxis_title="X",
    yaxis_title="Y",
    zaxis_title="Z",
    aspectmode='data'
))

# Show the interactive plot
fig.show()
